In [1]:
import opendssdirect as dss
import os
import pathlib
import numpy as np
import pandas as pd
import random
import sys
from scipy.stats import *
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import math
from matplotlib.animation import FuncAnimation
from celluloid import Camera

In [2]:
dss.__version__
dss.run_command('Redirect C:/Users/smartgrid_AI/Documents/kms/Hyundai/IEEE34BUSSYSTEM/ieee34Mod2.dss')
dss.run_command("New Energymeter.M1  Line.L1  1")

''

In [3]:
dss.Solution.Solve()
Ymatrixori = dss.Circuit.SystemY()
Ymatrix = np.array(Ymatrixori).reshape(138,-1)
Gm = Ymatrix[:, 0::2]
Bm = Ymatrix[:, 1::2]
dss.run_command("export Voltages")
Vthetmatrix = pd.read_csv("ieee34-2_EXP_VOLTAGES.csv", encoding='CP949', error_bad_lines=False)

In [4]:
def VTextract(Vthetmatrix):
    Vmat = np.zeros((Vthetmatrix.shape[0],3))
    Tmat = np.zeros((Vthetmatrix.shape[0],3))
    dis = np.zeros((Vthetmatrix.shape[0],3), dtype=int)
    for i in range(Vthetmatrix.shape[0]):
        for j in range(3):
            Vmat[i,j] = Vthetmatrix.iloc[i,4*j+3]*np.sqrt(3)
            Tmat[i,j] = Vthetmatrix.iloc[i,4*j+4]*np.pi/180
            if Vmat[i,j]!=0:
                dis[i,j]=1
    Vm = np.zeros(np.sum(dis))
    Tm = np.zeros(np.sum(dis))
    ena = 0
    for a in range(Vthetmatrix.shape[0]):
        for b in range(3):
            if dis[a,b]==1:
                Vm[ena]=Vmat[a,b]
                Tm[ena]=Tmat[a,b]
                ena = ena+1             
    return ena, Vm, Tm

def Sensecalc(ena, Vm, Tm):
    j1 = np.zeros((ena,ena))
    j2 = np.zeros((ena,ena))
    j3 = np.zeros((ena,ena))
    j4 = np.zeros((ena,ena))

    for a in range(ena):
        for b in range(ena):
            if a!=b:
                j1[a,b]=Vm[a]*Vm[b]*(Gm[a,b]*np.sin(Tm[a]-Tm[b])-Bm[a,b]*np.cos(Tm[a]-Tm[b]))
                j2[a,b]=Vm[a]*(Gm[a,b]*np.cos(Tm[a]-Tm[b])+Bm[a,b]*np.sin(Tm[a]-Tm[b]))
                j3[a,b]=-Vm[a]*Vm[b]*(Gm[a,b]*np.cos(Tm[a]-Tm[b])+Bm[a,b]*np.sin(Tm[a]-Tm[b]))
                j4[a,b]=Vm[a]*(Gm[a,b]*np.sin(Tm[a]-Tm[b])-Bm[a,b]*np.cos(Tm[a]-Tm[b]))
            if a==b:
                for c in range(ena):
                    if c!=a:
                        j1[a,b]=j1[a,b]-Vm[a]*Vm[c]*(Gm[a,c]*np.sin(Tm[a]-Tm[c])-Bm[a,c]*np.cos(Tm[a]-Tm[c]))
                        j2[a,b]=j2[a,b]+Vm[c]*(Gm[a,c]*np.cos(Tm[a]-Tm[c])+Bm[a,c]*np.sin(Tm[a]-Tm[c]))
                        j3[a,b]=j3[a,b]+Vm[a]*Vm[c]*(Gm[a,c]*np.cos(Tm[a]-Tm[c])+Bm[a,c]*np.sin(Tm[a]-Tm[c]))
                        j4[a,b]=j4[a,b]+Vm[c]*(Gm[a,c]*np.sin(Tm[a]-Tm[c])-Bm[a,c]*np.cos(Tm[a]-Tm[c]))
                    if c==a:
                        j2[a,b]=j2[a,b]+2*Vm[a]*Gm[a,b]
                        j4[a,b]=j4[a,b]-2*Vm[a]*Bm[a,b]                 
            if a<3 and b<3:
                j1[a,b]=0
                j2[a,b]=0
                j3[a,b]=0
                j4[a,b]=0      

    svq=np.linalg.inv(j4-np.matmul(j3,np.matmul(np.linalg.pinv(j1),j2)))
    svp=np.linalg.inv(j2-np.matmul(j1,np.matmul(np.linalg.pinv(j3),j4)))
    return svp, svq

allnodename = dss.Circuit.AllNodeNames()
allnode = np.array(allnodename)

def PVsize(num_pv):
    try:
        pv_size = np.array(list(map(int, input('각 PV의 크기를 입력하세요: ').split())))
        if pv_size.shape[0] != num_pv:
            print("PV의 개수에 맞게 입력하세요")
            PVsize()
    except:
        print("정수형으로 크기를 입력하세요")
        PVsize()
    return pv_size

def USERPV():
    try:
        pv_list = list(map(int, input('PV 모선 번호를 입력하세요: ').split()))
        pv_array = np.array(pv_list, dtype = int)
        num_pv = pv_array.shape[0]
        pv_busind = np.zeros(num_pv, dtype=int)
        nodelist = []
    
        for i in range(num_pv):
            for j in range(1,4):
                try:
                    x = np.where(allnode == str(pv_array[i])+'.'+str(j))[0][0]
                    nodelist.append(x)
                except:
                    print(str(pv_array[i])+"번 모선은 단상 모선입니다")
                    USERPV()
    except:
        print("PV 모선 번호가 잘못 입력되었습니다")
        USERPV()
    pv_size = PVsize(num_pv)
    return pv_array, num_pv, np.array(nodelist, dtype=int), pv_size

def checkvoltage(Vm):
    V2pu = np.array(Vm)
    for i in range(len(V2pu)):
        if V2pu[i]>60000:
            V2pu[i] = Vm[i]/69000
        elif V2pu[i]>20000:
            V2pu[i] = Vm[i]/24900
        else:
            V2pu[i] = Vm[i]/4160

    V2vio_u = (V2pu[3:]-uv>0)
    V2vio_l = (V2pu[3:]-lv<0)
    vio_node = V2vio_u+V2vio_l
    return vio_node, V2pu

In [5]:
# Example values - PV Bus  890 848 840 832 828
# Example values - PV Size 500 300 200 300 200 -> SUM: 1500 kW
pv_array, num_pv, pvnodeind, pv_size = USERPV()

PV 모선 번호를 입력하세요:  890 848 840 832 828
각 PV의 크기를 입력하세요:  500 300 200 300 200


In [6]:
ena, Vm, Tm = VTextract(Vthetmatrix)
Vmini = Vm
svp, svq = Sensecalc(ena, Vm, Tm)

In [7]:
for m in range(0,num_pv):
    if pv_array[m]!=0:
        if Vm[pvnodeind[3*m]]>50000:
            basekV = 69
        elif Vm[pvnodeind[3*m]]>15000:
            basekV = 24.9
        else:
            basekV = 4.16
        pv_kva_text="New PVSystem.PV"+str(m+1)+" conn=Wye phases=3 bus="+str(pv_array[m])+" kV="+str(basekV)+" irrad=1.0 Pmpp="+str(pv_size[m])+" temperature=25 kVA="+str(pv_size[m])+" WattPriority=True"
        print(pv_kva_text)
        dss.run_command(pv_kva_text)

New PVSystem.PV1 conn=Wye phases=3 bus=890 kV=4.16 irrad=1.0 Pmpp=500 temperature=25 kVA=500 WattPriority=True
New PVSystem.PV2 conn=Wye phases=3 bus=848 kV=24.9 irrad=1.0 Pmpp=300 temperature=25 kVA=300 WattPriority=True
New PVSystem.PV3 conn=Wye phases=3 bus=840 kV=24.9 irrad=1.0 Pmpp=200 temperature=25 kVA=200 WattPriority=True
New PVSystem.PV4 conn=Wye phases=3 bus=832 kV=24.9 irrad=1.0 Pmpp=300 temperature=25 kVA=300 WattPriority=True
New PVSystem.PV5 conn=Wye phases=3 bus=828 kV=24.9 irrad=1.0 Pmpp=200 temperature=25 kVA=200 WattPriority=True


In [8]:
uv = 1.05
lv = 0.92

In [9]:
dss.Solution.Solve()
dss.run_command("export Voltages")
Vthetmatrix = pd.read_csv("ieee34-2_EXP_VOLTAGES.csv", encoding='CP949', error_bad_lines=False)
_, Vm, _ = VTextract(Vthetmatrix)
_, V2pu = checkvoltage(Vm)

In [10]:
svp, svq = Sensecalc(ena, Vm, Tm)

In [11]:
delV = np.zeros(V2pu.shape)
for i in range(V2pu.shape[0]):
    if V2pu[i] > 1.05:
        delV[i] = V2pu[i]-1.05
    else:
        delV[i] = 0

In [25]:
delV

array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       5.40630818e-05, 0.00000000e+00, 0.00000000e+00, 7.49311638e-05,
       0.00000000e+00, 0.00000000e+00, 1.23623355e-04, 0.00000000e+00,
       0.00000000e+00, 1.83480608e-03, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 3.25383566e-03, 0.00000000e+00,
       0.00000000e+00, 4.17203127e-03, 3.60584606e-02, 2.90954772e-02,
       3.71157767e-02, 3.60584606e-02, 2.90954772e-02, 3.71157767e-02,
       3.59123840e-02, 2.90746091e-02, 3.71227328e-02, 3.49385402e-02,
       3.57106592e-02, 2.81633696e-02, 3.73453256e-02, 3.55089344e-02,
       2.72729981e-02, 3.75887866e-02, 2.11447380e-02, 1.00568304e-02,
       6.86401385e-03, 6.87096987e-03, 2.70643172e-02, 2.70643172e-02,
       3.54950224e-02, 2.72451740e-02, 3.75957426e-02, 3.54811103e-02,
       2.72103938e-02, 3.76096547e-02, 3.39160042e-02, 2.54087827e-02,
      

In [12]:
node3plist = []
node3indlist = []
for i in range(allnode.shape[0]-2):
    if allnode[i][:4] == allnode[i+1][:4] and allnode[i][:4] == allnode[i+2][:4]:
            node3plist.append(allnode[i][:-2])
            node3indlist.append([i, i+1, i+2])
node3p = np.array(node3plist)
node3yes = np.zeros(allnode.shape, dtype=int)

for i in range(node3p.shape[0]):
    for j in range(allnode.shape[0]):
        if node3p[i] == allnode[j][:-2]:
            node3yes[j] = 1
            
node3ind = np.array(node3indlist).reshape(-1)

In [13]:
qsensum3 = np.zeros(delV.shape[0])
for i in range(delV.shape[0]):
    qsensum3[i] = np.sum(np.multiply(delV, svq[:,i]))
qsensum = qsensum3[node3ind]
qsens = qsensum.reshape(-1,3)
qsensavg = np.mean(qsens, axis=1)
maxind = np.argmax(qsensavg)
qcomp_node = node3ind[maxind*3:maxind*3+3]
svq_comp = svq[:,qcomp_node]

In [14]:
if np.mean(Vm[qcomp_node])>50000:
    CbasekV = 69
elif np.mean(Vm[qcomp_node])>15000:
    CbasekV = 24.9
else:
    CbasekV = 4.16

In [15]:
cap_text = "New Capacitor.Cap1  Bus1="+allnode[qcomp_node][0][:-2]+" Phases=3 kVAR="+str(0)+" kV="+str(CbasekV)
print(cap_text)
dss.run_command(cap_text)

New Capacitor.Cap1  Bus1=840 Phases=3 kVAR=0 kV=24.9


''

In [20]:
LB = 0
UB = sum(pv_size)*2
N = 4

In [21]:
X = np.zeros(N)
for i in range(N):
    X[i] = np.around((UB-LB)/N*(i+1))
X

array([ 750., 1500., 2250., 3000.])

In [22]:
def cap_change(capsize):
    cap_change_text = "Capacitor.cap1.kvar=-"+str(capsize)
    dss.run_command(cap_change_text)
    #print(dss.utils.capacitors_to_dataframe())
    dss.Solution.Solve()
    dss.run_command("export Voltages")
    Vthetmatrix = pd.read_csv("ieee34-2_EXP_VOLTAGES.csv", encoding='CP949', error_bad_lines=False)
    _, Vm, _ = VTextract(Vthetmatrix)
    _, V2pu = checkvoltage(Vm)
    return Vm, V2pu

def capablecapfind(X):
    regcapable = np.zeros(N, dtype=int)
    for i in range(N):
        Vm, V2pu = cap_change(X[i])
        Violation,_ = checkvoltage(Vm)
        if sum(Violation) == 0:
            regcapable[i] = 1
    return regcapable

def caprefind(regcapable, X, LB, UB):
    k=N-1
    for i in range(N-1):
        if regcapable[i] == 0 and regcapable[i+1] == 1:
            k=i
    if k==0:
        UB = X[k]
    elif k==N-1:
        LB = X[k]
    else:
        LB = X[k]
        UB = X[k+1]
    for i in range(N):
        X[i] = (UB-LB)/N*(i+1)+LB
    
    X = np.around(X)
    print(LB, UB, X)
    return X, LB, UB

In [23]:
while(1):
    regcapable = capablecapfind(X)
    X, LB, UB = caprefind(regcapable, X, LB, UB)
    if X[0]==X[-1]:
        break
print(X)

1500.0 2250.0 [1688. 1875. 2062. 2250.]
1500.0 1688.0 [1547. 1594. 1641. 1688.]
1688.0 1688.0 [1688. 1688. 1688. 1688.]
[1688. 1688. 1688. 1688.]
